## <h1><center> Предсказание "виральности комментария" с помощью методов машинного обучения</center></h1>

[1. Введение](#introduction)

[2.1. Сбор данных](#data-collection)

[2.2. Импорт библиотек и данных](#data-import)

[2.3. Предобработка данных](#data-preprocessing)

## 1 Введение <a class="anchor" id="introduction"></a>

Обработка естественного языка (Natural Language Processing, NLP) — направление машинного обучения, изучающее анализ и синтез текстов на естественном языке. На данный момент NLP — одна из самых важных сфер машинного обучения в том числе благодаря появлению BERT и ChatGPT.

Среди применений NLP можно выделить:

- Классификация текстов
- Извлечение информации и ключевых слов из текстов 
- Генерация текст-текст
- Машинный перевод и т.д.

В рамках этого проекта я буду использовать NLP методы для того, чтобы определять *сколько лайков может получить комментарий на YouTube (Fox News)* на основе их содержания, а также дополнительных метаданных.

**Задачи**:
- Сбор и предобработка данных
- Разведовательный анализ данных (Explanatory Data Analysis, EDA)
- Обучение модели
- Оценка результатов

**Данные**. В качестве данных для обучения модели я буду использовать комментарии с канала Fox News на Youtube. Я соберу их вручную с помощью Google API.

**Модель**. В рамках этого проекта я хочу получить опыт c SOTA архитектурой Tranformers. Про выбор модели подробнее в соответствующем разделе.

### 2.1 Сбор данных <a class="anchor" id="data-collection"></a>

В качестве данных для обучения будут использоваться комментарии с YouTube-канала одного из самых популярных Американских СМИ — Fox News. Результаты полученные с помощью модели, обученной на этих данных, конечно, не могут использоваться где угодно, однако, в этой работе, я хочу, в первую очередь, получить практический опыт реализации NLP моделей.

Итак, для сбора данных я использовал *YouTube Data API v3*.

В репозитории проекта на GitHub имеются два Python скрипта — *video_parser.py* и *comments_parser.py*.

***video_parser.py*** — получает информацию о 50 самых популярных видео с канала в виде словаря
```json
{
"videoId": "UqE8IYUXsBs",
"videoPublishedAt": "2025-07-18T15:45:02Z"
}
```

***comments_parser.py*** — получает 100 популярных комментариев к каждому, полученному предыдущим скриптом видео. И формирует финальный датасет

| commentId                      | videoId      | textOriginal                                                                 | likeCount | videoPublishedAt       | commentPublishedAt    | commentUpdatedAt      |
|--------------------------------|--------------|-----------------------------------------------------------------------------|-----------|------------------------|-----------------------|-----------------------|
| UgwwQfnW75VHyFU8oCh4AaABAg     | UqE8IYUXsBs  | Read more: https://www.foxnews.com/us/feds-california-home-depot-raid-nabs... | 44        | 2025-07-18T15:45:02Z  | 2025-07-18T17:18:07Z | 2025-07-18T17:18:07Z |
| UgwL5M-S_ZkgZP-TaPJ4AaABAg     | UqE8IYUXsBs  | Start removing these judges and politicians that allow this mess in the first place! | 2315      | 2025-07-18T15:45:02Z  | 2025-07-18T16:50:39Z | 2025-07-18T16:50:39Z |
| UgyCiXsqEo-3mBy_fUx4AaABAg     | UqE8IYUXsBs  | He's been arrested 67 times, he shouldn't be here                            | 1488      | 2025-07-18T15:45:02Z  | 2025-07-18T16:51:11Z | 2025-07-18T16:51:11Z |

### 2.2 Импорт библиотек и данных <a class="anchor" id="data-import"></a>

In [19]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sentencepiece import SentencePieceTrainer, SentencePieceProcessor

In [8]:
data = pd.read_csv(
    filepath_or_buffer='parsed_content/comments.csv',
)

data.head()

,commentId,videoId,textOriginal,likeCount,videoPublishedAt,commentPublishedAt,commentUpdatedAt
0,UgwwQfnW75VHyFU8oCh4AaABAg,UqE8IYUXsBs,Read more: https://www.foxnews.com/us/feds-cal...,44,2025-07-18T15:45:02Z,2025-07-18T17:18:07Z,2025-07-18T17:18:07Z
1,UgwL5M-S_ZkgZP-TaPJ4AaABAg,UqE8IYUXsBs,Start removing these judges and politicians th...,2315,2025-07-18T15:45:02Z,2025-07-18T16:50:39Z,2025-07-18T16:50:39Z
2,UgyCiXsqEo-3mBy_fUx4AaABAg,UqE8IYUXsBs,"He's been arrested 67 times, he shouldn't be here",1488,2025-07-18T15:45:02Z,2025-07-18T16:51:11Z,2025-07-18T16:51:11Z
3,UgyfSvY23tYFDfpzbvx4AaABAg,UqE8IYUXsBs,He sliced the tires.\n They left that informat...,1117,2025-07-18T15:45:02Z,2025-07-18T23:52:04Z,2025-07-18T23:52:04Z
4,Ugxu4iFba7FZ06CfMK54AaABAg,UqE8IYUXsBs,67 arrests since 1986!! Jail the judges who f...,70,2025-07-18T15:45:02Z,2025-07-19T17:46:43Z,2025-07-19T17:46:43Z


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   commentId           5000 non-null   object
 1   videoId             5000 non-null   object
 2   textOriginal        4999 non-null   object
 3   likeCount           5000 non-null   int64 
 4   videoPublishedAt    5000 non-null   object
 5   commentPublishedAt  5000 non-null   object
 6   commentUpdatedAt    5000 non-null   object
dtypes: int64(1), object(6)
memory usage: 273.6+ KB


Итак, датасет содержит 5000 комментариев со следующими характеристиками:

- commentId — уникальный идентификатор комментария
- videoId — уникальный идентификатор видео
- textOriginal — полный текст комментария
- likeCount — количество лайков под комментарием (*целевая переменная*)
- videoPublishedAt — timestamp публикации видео
- commentPublishedAt — timestamp публикации комментария
- commentUpdatedAt - timestamp редактирования комментария

На этапе предобработки данных нужно будет обработать пропущенное значение в textOriginal, привести типы данных, а также сделать временные столбцы более информативными, нормализировать данные, токенизировать комментарии.

### 2.3 Предобработка данных <a class="anchor" id="data-preprocessing"></a>

Следующим этапом я выбрал предобаботку данных, потому что она будет включать в себя feature engineering и в EDA я бы хотел увидеть распределения и характеристики всех фичей перед обучением модели. Если на этапе EDA обнаружаться выбросы или еще какие-то проблемы с данными они будут обработаны позже (после EDA).

Начну с того, что удалю пропуски и дубликаты

In [10]:
data = data.drop_duplicates(subset='textOriginal')

data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4971 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   commentId           4971 non-null   object
 1   videoId             4971 non-null   object
 2   textOriginal        4970 non-null   object
 3   likeCount           4971 non-null   int64 
 4   videoPublishedAt    4971 non-null   object
 5   commentPublishedAt  4971 non-null   object
 6   commentUpdatedAt    4971 non-null   object
dtypes: int64(1), object(6)
memory usage: 310.7+ KB


Как можно заметить было удалено 29 одинаковых комментариев.

Теперь удалю пропущенные значения

In [11]:
data = data.dropna()

data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4970 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   commentId           4970 non-null   object
 1   videoId             4970 non-null   object
 2   textOriginal        4970 non-null   object
 3   likeCount           4970 non-null   int64 
 4   videoPublishedAt    4970 non-null   object
 5   commentPublishedAt  4970 non-null   object
 6   commentUpdatedAt    4970 non-null   object
dtypes: int64(1), object(6)
memory usage: 310.6+ KB


Также стоит удалить комментарии, которые редактировались, потому что мы не можем знать какую именно версию комментария пользователи одобрили лайком.

In [12]:
data = data.drop(data[data['commentPublishedAt'] != data['commentUpdatedAt']].index)

data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4755 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   commentId           4755 non-null   object
 1   videoId             4755 non-null   object
 2   textOriginal        4755 non-null   object
 3   likeCount           4755 non-null   int64 
 4   videoPublishedAt    4755 non-null   object
 5   commentPublishedAt  4755 non-null   object
 6   commentUpdatedAt    4755 non-null   object
dtypes: int64(1), object(6)
memory usage: 297.2+ KB


Теперь, поскольку в столбцах commentPublishedAt и commentUpdatedAt находятся одинаковые значения стоит удалить последний.

In [13]:
data = data.drop(['commentUpdatedAt'], axis=1)

Теперь датасет содержит 4755 уникальных комментариев.

Следующим шагом я преобразую типы данных, для дальнейшей корректной работы с фичами.

In [14]:
data['videoPublishedAt'] = pd.to_datetime(
    data['videoPublishedAt'], 
    format='%Y-%m-%dT%H:%M:%SZ', 
    utc=True
)

data['commentPublishedAt'] = pd.to_datetime(
    data['commentPublishedAt'], 
    format='%Y-%m-%dT%H:%M:%SZ', 
    utc=True
)

data = data.astype({
    'commentId' : 'string',
    'videoId' : 'string',
    'textOriginal' : 'string',
    'likeCount' : 'int64',
    'videoPublishedAt' : 'int64',
    'commentPublishedAt' : 'int64',
})

data.dtypes

commentId             string[python]
videoId               string[python]
textOriginal          string[python]
likeCount                      int64
videoPublishedAt               int64
commentPublishedAt             int64
dtype: object

Данные приведены к нормальным типам теперь над ними можно безопасно проводить различные операции.

Начну с того, что создам новый столбец, который будет хранить время, пройденное с начала публикации до написания комментария.

In [15]:
data['commentDelay'] = data['commentPublishedAt'] - data['videoPublishedAt']

data = data.drop(['videoPublishedAt', 'commentPublishedAt'], axis=1)

data.head()

,commentId,videoId,textOriginal,likeCount,commentDelay
0,UgwwQfnW75VHyFU8oCh4AaABAg,UqE8IYUXsBs,Read more: https://www.foxnews.com/us/feds-cal...,44,5585000000000
1,UgwL5M-S_ZkgZP-TaPJ4AaABAg,UqE8IYUXsBs,Start removing these judges and politicians th...,2315,3937000000000
2,UgyCiXsqEo-3mBy_fUx4AaABAg,UqE8IYUXsBs,"He's been arrested 67 times, he shouldn't be here",1488,3969000000000
3,UgyfSvY23tYFDfpzbvx4AaABAg,UqE8IYUXsBs,He sliced the tires. They left that informati...,1117,29222000000000
4,Ugxu4iFba7FZ06CfMK54AaABAg,UqE8IYUXsBs,67 arrests since 1986!! Jail the judges who f...,70,93701000000000


Теперь, так как архитектура целевой модели использует операции с матрицами и алгебру с числами, стоит стандартизировать числовые значения.

In [16]:
data[['likeCount', 'commentDelay']] = MinMaxScaler().fit_transform(data[['likeCount', 'commentDelay']])

data.head()

,commentId,videoId,textOriginal,likeCount,commentDelay
0,UgwwQfnW75VHyFU8oCh4AaABAg,UqE8IYUXsBs,Read more: https://www.foxnews.com/us/feds-cal...,0.019006,0.016787
1,UgwL5M-S_ZkgZP-TaPJ4AaABAg,UqE8IYUXsBs,Start removing these judges and politicians th...,1.000000,0.013586
2,UgyCiXsqEo-3mBy_fUx4AaABAg,UqE8IYUXsBs,"He's been arrested 67 times, he shouldn't be here",0.642765,0.013648
3,UgyfSvY23tYFDfpzbvx4AaABAg,UqE8IYUXsBs,He sliced the tires. They left that informati...,0.482505,0.062695
4,Ugxu4iFba7FZ06CfMK54AaABAg,UqE8IYUXsBs,67 arrests since 1986!! Jail the judges who f...,0.030238,0.187929


Теперь пришло время к важному этапу в реализции моделей обрабатывающих естественный язык — преобразование текстов в числовые представения — embeddings.

Для начала необходимо получить токены из наших данных. NLP модели, которые оперируют текстом, испольщуют понятие токена.

Токен — это единица текста, которую спсобен понимать алгоритм. В этой работе я буду использовать современный токенизатор основанный на Byte Pair Encoding (BPE) — Google SentencePiece Tokenizer. Этот метод хорошо подходит под выполняемую задачу, потому что:

- люди совершают ошибки при написании коментарии и subword tokenization лучше справляется с подобными ошибками, чем другие способы токенизации
- люди используют сленг и неформальную лексику, BPE может улавливать это
- может обрабатывать слова не содержащиеся в исходных данных.

Начнем с создания обучающих данных для токенизатора.

In [17]:
with open('all_comments_text.txt', 'w', encoding='utf-8') as file:
    for text in data['textOriginal']:
        file.write(text + '\n')

In [21]:
SentencePieceTrainer.train(
    '--input=all_comments_text.txt --model_prefix=m_bpe --vocab_size=2000 --model_type=bpe'
)

m_bpe_processor = SentencePieceProcessor()
m_bpe_processor.load('m_bpe.model')

sentencepiece_trainer.cc(178) LOG(INFO) Running command: --input=all_comments_text.txt --model_prefix=m_bpe --vocab_size=2000 --model_type=bpe
sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: all_comments_text.txt
  input_format: 
  model_prefix: m_bpe
  model_type: BPE
  vocab_size: 2000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2


True

er.cc(268) LOG(INFO) Added: freq=53 size=700 all=12536 active=1564 piece=TH
bpe_model_trainer.cc(159) LOG(INFO) Updating active symbols. max_freq=53 min_freq=20
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=51 size=720 all=12699 active=1146 piece=ani
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=50 size=740 all=12822 active=1269 piece=▁Kai
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=49 size=760 all=12886 active=1333 piece=▁agents
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=47 size=780 all=13009 active=1456 piece=OC
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=46 size=800 all=13119 active=1566 piece=▁another
bpe_model_trainer.cc(159) LOG(INFO) Updating active symbols. max_freq=46 min_freq=18
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=44 size=820 all=13229 active=1111 piece=man
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=43 size=840 all=13339 active=1221 piece=▁vote
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=42 size=860 all=13394 active=1276 piece=erson
bpe_m